In [1]:
from torch.utils.tensorboard import SummaryWriter   
import pandas as pd,os
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict, KFold
from tqdm import tqdm
import matplotlib.pyplot as plt
from skorch import NeuralNetClassifier
from sklearn.metrics import accuracy_score
# model_type = 'LSTM'
# model_type = 'MLP'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [2]:
# data_train, ans_train = np.load('./dataset4/X_train_class0_time.npy'), np.load('./dataset4/y_train_class0_time.npy', allow_pickle=True)
# data_test, ans_test = np.load('./dataset4/X_test_class0_time.npy'), np.load('./dataset4/y_test_class0_time.npy', allow_pickle=True)

# data_train, ans_train = np.load('./dataset4/X_train_class10_time.npy'), np.load('./dataset4/y_train_class10_time.npy', allow_pickle=True)
# data_test, ans_test = np.load('./dataset4/X_test_class10_time.npy'), np.load('./dataset4/y_test_class10_time.npy', allow_pickle=True)

# data_train, ans_train = np.load('./dataset4/X_train_class50_time.npy'), np.load('./dataset4/y_train_class50_time.npy', allow_pickle=True)
# data_test, ans_test = np.load('./dataset4/X_test_class50_time.npy'), np.load('./dataset4/y_test_class50_time.npy', allow_pickle=True)

data_train, ans_train = np.load('./dataset4/X_train_class100_time.npy'), np.load('./dataset4/y_train_class100_time.npy', allow_pickle=True)
data_test, ans_test = np.load('./dataset4/X_test_class100_time.npy'), np.load('./dataset4/y_test_class100_time.npy', allow_pickle=True)



label_train = np.unique(ans_train, return_counts=False,return_index=False,return_inverse=False)
label_test = np.unique(ans_test, return_counts=False,return_index=False,return_inverse=False)
print(len(label_train), len(label_test))

7 7


In [3]:
def label2value(ans, data):
    X = []
    y = []
    
    for idi,i in enumerate(ans):
        pop = False
        for idj, j in enumerate(label_train):
            if j==i:
                pop = True
                y.append(idj)
        if pop==True:
            X.append(data[idi])
    return np.array(X), np.array(y)
X_train, y_train = label2value(ans_train, data_train)
X_test, y_test = label2value(ans_test, data_test)
print(len(np.unique(y_train, return_counts=False,return_index=False,return_inverse=False)),len(np.unique(y_test, return_counts=False,return_index=False,return_inverse=False)))
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

7 7
(1696, 177848) (1696,) (566, 177848) (566,)


In [5]:
class RNN(nn.Module):
    def __init__(self, class_n, Input_Size):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(
            input_size=Input_Size,
            hidden_size=128,
            num_layers=1,
            batch_first=True,
        )

        self.out = nn.Linear(128, class_n)
    def forward(self, x):
        r_out, (h_c, h_h) = self.rnn(x, None)
        out = self.out(r_out[:, -1, :])
        return out
    
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.normal_(m.weight.data, 0, 0.01)
                # m.weight.data.normal_(0,0.01)
                m.bias.data.zero_()

class MLP(nn.Module):
    def __init__(self, num_classes, input_size):
        super(MLP,self).__init__()
        self.linear1 = nn.Linear(in_features=input_size, out_features=1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.dt1 = nn.Dropout(0.25)
        self.linear2 = nn.Linear(in_features=1024, out_features=256)
        self.bn2 = nn.BatchNorm1d(256)
        self.dt2 = nn.Dropout(0.25)
        self.linear3 = nn.Linear(in_features=256, out_features= num_classes)
        
    def forward(self, x):
        x = self.bn1(self.linear1(x))
        x = F.relu(x)
        # x = self.dt1(x)
        x = self.bn2(self.linear2(x))
        x = F.relu(x)
        # x = self.dt2(x)
        x = self.linear3(x)
        x = F.softmax(x, dim=1)
        return x
        
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                torch.nn.init.normal_(m.weight.data, 0, 0.01)
                # m.weight.data.normal_(0,0.01)
                m.bias.data.zero_()

In [6]:
LR = 0.0001
epochs = 380
batch_size = 128
k_cv = KFold(n_splits=10, random_state=42, shuffle=True) 

device = ('cuda' if torch.cuda.is_available() else 'cpu')
if hasattr(torch.cuda, 'empty_cache'):
        torch.cuda.empty_cache()

def accuracy(predictions, labels):
    classes = torch.argmax(predictions, dim=1)
    return torch.mean((classes == labels).float())

def trainer(idx, train_index, valid_index):
    
    X_, x_ = X_train[train_index], X_train[valid_index]
    Y_, y_ = y_train[train_index], y_train[valid_index]
    print(X_.shape, x_.shape)
    train_zip = TensorDataset(torch.tensor(X_), torch.tensor(Y_))
    valid_zip = TensorDataset(torch.tensor(x_), torch.tensor(y_))
    train_loader = DataLoader(dataset=train_zip, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(dataset=valid_zip, batch_size=batch_size, shuffle=False)
    train_Ls = []
    valid_Ls = []
    # Starting trainer
    Input_Size = np.array(X_train).shape[1]
    if model_type=='LSTM':
        model = RNN(max(y_train)+1, Input_Size).to(device)
    else:
        model = MLP(max(y_train)+1, Input_Size).to(device)
    # model.initialize_weights()
    loss_fun = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    min_valid_loss = np.inf
    for epoch in tqdm(range(epochs)):
            correct = 0
            for step, (b_x, b_y) in enumerate(train_loader):
                # Forward pass
                model.train()
                train_loss = 0.0
                if model_type=='LSTM':
                    b_x_, b_y_= (torch.unsqueeze(b_x, 1)).data.type(torch.FloatTensor).to(device), b_y.to(device)
                else:
                    b_x_, b_y_= b_x.float().to(device), b_y.to(device)
                optimizer.zero_grad()
                r_out = model(b_x_)
                loss = loss_fun(r_out, b_y_)
                loss.backward()
                optimizer.step()
                train_loss = loss.item()*b_x_.size(0)
                correct = correct + accuracy(r_out, b_y_)
                del b_x_, b_y_
                torch.cuda.empty_cache()
            train_acc = 100 * correct / len(train_loader)
            train_writer.add_scalar('accuracy', train_acc, epoch)
            train_writer.add_scalar('losses', train_loss, epoch)
            train_Ls.append(train_loss)
            # Validation
            correct2=0
            with torch.no_grad():
                valid_loss = 0.0
                model.eval()
                for step, (b_x, b_y) in enumerate(valid_loader):
                    if model_type=='LSTM':
                        b_x_, b_y_= (torch.unsqueeze(b_x, 1)).data.type(torch.FloatTensor).to(device), b_y.to(device)
                    else:
                        b_x_, b_y_= b_x.float().to(device), b_y.to(device)
                    r_out = model(b_x_)
                    loss = loss_fun(r_out, b_y_)
                    valid_loss = loss.item()*b_x_.size(0)
                    v_pred = torch.argmax(r_out, dim=1)
                    # print(accuracy(r_out, b_y_))
                    correct2  = correct2 + accuracy(r_out, b_y_)
                    del b_x_, b_y_
                    torch.cuda.empty_cache()

                if min_valid_loss > valid_loss:
                    min_valid_loss = valid_loss
                    torch.save({'state_dict': model.state_dict()}, f'./pth/{model_type + "_"+str(idx+1)}_saved_model_class{max(y_train)+1}.pth.tar')

            valid_acc = 100 * correct2 / len(valid_loader)
            valid_writer.add_scalar('accuracy', valid_acc, epoch)
            valid_writer.add_scalar('losses', valid_loss, epoch)
            valid_Ls.append(valid_loss)     
    return [train_Ls, valid_Ls]

In [7]:
import matplotlib.pyplot as plt
md_type_stack= ['MLP', 'LSTM']
for j in md_type_stack:
    model_type = j
    for idx,(train_index, valid_index) in enumerate(k_cv.split(X_train)):
        log_dir =os.path.join('./path/to/log/',f'{model_type}_{max(y_train)+1}_{idx+1}')
        train_writer = SummaryWriter(log_dir=log_dir+'/train')
        valid_writer = SummaryWriter(log_dir=log_dir+'/valid')
        loss_stack = trainer(idx, train_index, valid_index)
        
        fig = plt.figure(figsize=(15,5))
        plt.plot(loss_stack[0], label='train')
        plt.plot(loss_stack[1], label='test')
        plt.title(f'Kfold - {idx+1} {model_type} Class {max(y_train)+1}')
        plt.legend()
        if not os.path.exists(f'Results/'):
            os.makedirs(f'Results/')
        plt.savefig(f'Results/Kfold - {idx+1} {model_type} Class {max(y_train)+1} Training Plot.jpg')
        plt.cla()
        plt.clf()
        plt.close()

(1526, 177848) (170, 177848)


100%|██████████| 380/380 [11:32<00:00,  1.82s/it]


(1526, 177848) (170, 177848)


100%|██████████| 380/380 [11:40<00:00,  1.84s/it]


(1526, 177848) (170, 177848)


...

100%|██████████| 380/380 [09:48<00:00,  1.55s/it]


(1527, 177848) (169, 177848)


100%|██████████| 380/380 [10:59<00:00,  1.74s/it]


(1527, 177848) (169, 177848)


100%|██████████| 380/380 [09:05<00:00,  1.44s/it]


In [8]:
if model_type=='LSTM':
    net = NeuralNetClassifier(module=RNN, train_split=None)
else:
    net = NeuralNetClassifier(module=MLP, train_split=None)


# net = NeuralNetClassifier(module=RNN, train_split=None)
# y_pred = cross_val_predict(net, X_train, y_train, cv=k_cv)

In [9]:
# # model loadings
# if model_type=='LSTM':
#         model = RNN(max(y_train)+1, Input_Size).to(device)
# else:
#         model = MLP(max(y_train)+1, Input_Size).to(device)
# # checkpoint = torch.load(f'./pth/{model_type}_saved_model_50_epoch-1000.pth.tar', map_location='cpu')
# checkpoint = torch.load(f'./pth/{model_type}_saved_model_50_epoch-1000.pth.tar')
# model.load_state_dict(checkpoint['state_dict'])
# print(model)

In [10]:
# DNA_list = ['-', 'A', 'C', 'G', 'N', 'T']
# print(X_train.shape[1])

In [11]:
# fn = pd.read_csv('../lstm covid19/feature_name.csv')

# fn = np.array(fn.values)
# fn = fn.flatten()

In [12]:
# # def index2text(value_data):
# #     text=[]
# #     for id1, n1 in enumerate(np.array(value_data)):
# #         for id2, n2 in enumerate(feature_text):
# #             if n1==id2:
# #                 add_ = str(n2)+'_'+str(id1)
# #                 text.append(add_)
# #     return text
# import matplotlib.pyplot as plt
# import shap
# shap.initjs()
# # class_n = 3 #0-7
# # for f_id, class_n in enumerate(label_train):
# f_id = 0
# class_n = label_test[f_id]
# batch = next(iter(valid_loader))
# value, ans = batch
# new_value = []
# # for id, i in enumerate(ans):
#     # if i==f_id:
#         # new_value.append(np.array(value[id]))

# # background = torch.tensor(new_value)
# background = value
# if model_type=='LSTM':
#     background_ex = (torch.unsqueeze(background, 1)).data.type(torch.FloatTensor)
# else:
#     background_ex = background.data.type(torch.FloatTensor)

# print('Test Data Shape ',value.shape)
# print('Extract Data Shape ', background.shape)
# print('Squeeze Data Shape ', background_ex.shape)

# with torch.no_grad():
#     e = shap.DeepExplainer(model.cpu(), background_ex.cpu())
# shap_values = e.shap_values(background_ex.cpu())

# test_stack = []
# # for i in range(len(new_value)):
# #     temp = index2text(new_value[i])
# #     test_stack.append(temp)
# test_stack = np.array(value)
# # test_stack = np.array(test_stack)
# print('Ground Turth Data Shape ', test_stack.shape)
# if model_type=='LSTM':
#     shap_values_ = np.squeeze(np.array(shap_values),axis=-2)
# else:
#     shap_values_ = np.array(shap_values)

# print('Shap Values Shape ', shap_values_.shape)



In [13]:
# f_id = 6
# class_n = label_test[f_id]
# print(shap_values_[f_id].shape, test_stack.shape)
# # for f_id, class_n in enumerate(label_train):
# shap.summary_plot(list(shap_values_), test_stack, plot_type="bar", class_names=label_test, feature_names=fn, show=False)
# fig = plt.gcf()
# fig.set_figheight(12)
# fig.set_figwidth(14)
# ax = plt.gca()
# ax.set_ylabel('Feature', fontsize=16)
# leg = ax.legend()

# plt.title(f'{model_type} Multiclass', fontsize=20)
# # plt.show()
# plt.rcParams["figure.figsize"] = (10,20)
# plt.savefig(f'/ssd1/報告//Images//DNA/{model_type}//{model_type} multiclass-100-output.jpg', dpi=300)
# # shap.summary_plot(shap_values, X)